In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [3]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

produto = "tilápia"
url = f"https://www.carrefour.com.br/busca?term={produto}"
driver.get(url)
time.sleep(5)


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Configurações do navegador
options = Options()
options.add_argument("--headless")  # sem abrir a janela do navegador
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

try:
    print("Iniciando o driver...")
    driver = webdriver.Chrome(options=options)  # ou especifique o caminho: executable_path='...'
    
    url = "https://www.carrefour.com.br/busca?term=tilapia"
    print(f"Acessando URL: {url}")
    driver.get(url)

    time.sleep(5)
    print("Página carregada. Título:", driver.title)

    # Testar se carregou algum conteúdo
    body = driver.find_element(By.TAG_NAME, "body")
    print("Conteúdo da página (parcial):", body.text[:500])

except Exception as e:
    print("Erro ao acessar o site:", e)

finally:
    driver.quit()


Iniciando o driver...
Acessando URL: https://www.carrefour.com.br/busca?term=tilapia
Página carregada. Título: Mercado Carrefour | Ofertas de Supermercado Delivery | Carrefour
Conteúdo da página (parcial): Utilizamos cookies próprios e de terceiros para melhorar a navegação, desempenho e segurança do site, exibir anúncios e obter estatísticas. Ao clicar em "Prosseguir com todos", você concorda com o armazenamento de informações para tais finalidades, sendo possível adequar o uso dos cookies de acordo com suas preferências em "Personalizar cookies". Acesse nossa Política de Privacidade.
Prosseguir com todos
Personalizar Cookies
SHOPPING
MERCADO
DROGARIA
SERVIÇOS
Nossas lojas
Atendimento
Atendimento


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Configuração do Chrome em modo headless
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)

try:
    url = "https://www.carrefour.com.br/busca?term=tilapia"
    driver.get(url)

    # Espera até 20 segundos por algum botão com "Prosseguir" ou similar
    try:
        cookie_btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((
                By.XPATH, "//button[contains(text(), 'Prosseguir')]"
            ))
        )
        cookie_btn.click()
        print("Botão de cookies clicado.")
    except:
        print("Botão de cookies não encontrado. Continuando mesmo assim...")

    time.sleep(5)

    produtos = driver.find_elements(By.CLASS_NAME, "product-card__description")
    print(f"Número de produtos encontrados: {len(produtos)}")

    dados = []
    for produto in produtos:
        try:
            nome = produto.find_element(By.CLASS_NAME, "product-card__name").text
            preco = produto.find_element(By.CLASS_NAME, "sales-price").text
            dados.append({"nome": nome, "preco": preco})
        except:
            continue

    df = pd.DataFrame(dados)
    print(df)

finally:
    driver.quit()


Botão de cookies não encontrado. Continuando mesmo assim...
Número de produtos encontrados: 0
Empty DataFrame
Columns: []
Index: []


In [9]:
import requests
import pandas as pd

termo = "tilapia"
url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://mercado.carrefour.com.br/",
    "Origin": "https://mercado.carrefour.com.br",
    "Connection": "keep-alive"
}

response = requests.get(url, headers=headers)

if response.status_code in [200, 206]:
    produtos = response.json()
    dados = []
    for item in produtos:
        try:
            nome = item.get('productName', '')
            preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
            dados.append({"nome": nome, "preco": preco})
        except Exception as e:
            print(f"Erro ao processar item: {e}")
    
    df = pd.DataFrame(dados)
    print(df)
    df.to_csv("precos_carrefour.csv", index=False)
else:
    print(f"Erro na requisição: {response.status_code}")
    print("Resposta:", response.text[:500])


                                                nome  preco
0             Filé de Tilápia Fresco Carrefour 500 g  64.89
1  Filé de Tilápia sem Espinha sem Pele Congelado...  65.99
2   Filé de Tilápia a Milanesa Congelado Swift 500 g  42.25
3  Filé de Tilápia Congelado Carrefour Special Sa...  21.75
4  Quadritos de Peixe Empanado Costa Sul Pacote 400g  10.29
5             Carne Moída Incrível 100% Vegetal 200g  19.59
6                Filé De Tilápia           Qualyfish   0.00
7                Filé De Tilápia           Qualyfish   0.00
8                Filé De Tilápia           Qualyfish   0.00
9                Filé De Tilápia           Frescatto   0.00


In [10]:
import requests
import pandas as pd

def get_region_info(lat, long):
    url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&geoCoordinates={long},{lat}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    }

    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        data = r.json()
        if isinstance(data, list) and len(data) > 0:
            return {
                "regionId": data[0].get("regionId"),
                "addressId": data[0].get("addressId"),
                "loja_nome_api": data[0].get("friendlyName", ""),
                "cidade_api": data[0].get("city", ""),
                "estado_api": data[0].get("state", ""),
                "cep_api": data[0].get("postalCode", "")
            }
    return {}

# Carregar sua planilha
df = pd.read_excel("lista_lojas.xlsx")

# Aplicar a função para obter dados por loja
df_region = df.copy()
region_info = df_region.apply(lambda row: pd.Series(get_region_info(row['lat'], row['long'])), axis=1)

# Unir os dados
df_final = pd.concat([df_region, region_info], axis=1)

# Salvar resultado
df_final.to_csv("lojas_com_regionId.csv", index=False)


In [11]:
df_final

,estado,cidade/UF,loja,lat,long
0,AL,Maceió/AL,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739
1,AM,Manaus/AM,Hipermercado Manaus AdrianópolisAdrianopolis,-3.10513,-60.01123
2,NaN,NaN,Hipermercado Manaus ShoppingChapada,-3.09326,-60.02191
3,NaN,NaN,Hipermercado Ponta NegraDom Pedro,-3.09195,-60.04980
4,BA,Itabuna/BA,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272
...,...,...,...,...,...
112,NaN,NaN,Hipermercado InterlagosCampo Grande,-23.67753,-46.67525
113,NaN,NaN,Hipermercado LimãoLimao,-23.51228,-46.67499
114,NaN,NaN,Hipermercado PamplonaJd Pauilsta,-23.57010,-46.66136
115,NaN,NaN,Hipermercado PinheirosVila Gomes,-23.62417,-46.71079


In [ ]:
df_final.to_csv("lojas_com_regionId.csv", index=False)


In [17]:
# Coleta de Preços Carrefour por loja (Tilápia)
import pandas as pd
import requests
from datetime import datetime

# Carregar a planilha com coordenadas
caminho_arquivo = "lista_lojas.xlsx"  # ajuste conforme local do seu arquivo
lojas = pd.read_excel(caminho_arquivo)

# Função para obter o regionId

def get_region_id(lat, long):
    url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&geoCoordinates={long},{lat}"
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                return data[0].get("regionId")
    except Exception as e:
        print(f"Erro ao obter regionId: {e}")
    return None

# Função para buscar produtos com regionId

def buscar_produtos(region_id, termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    cookies = {"vtex_segment": f'{{"channel":"1","regionId":"{region_id}"}}'}
    try:
        r = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Teste com as 5 primeiras lojas
amostra = lojas.head(5)
registros = []

for _, row in amostra.iterrows():
    lat, long = row['lat'], row['long']
    loja_nome, cidade, estado = row['loja'], row['cidade/UF'], row['estado']
    region_id = get_region_id(lat, long)
    print(f"Loja: {loja_nome} | Cidade: {cidade} | Estado: {estado}")
    print(f"RegionId: {region_id}")
    if region_id:
        produtos = buscar_produtos(region_id, termo="peixe")  # trocado de tilapia para peixe
        print(f"Produtos encontrados: {len(produtos)}")
        for item in produtos:
            try:
                nome = item.get("productName", "")
                preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
                registros.append({
                    "produto": nome,
                    "preco": preco,
                    "loja": loja_nome,
                    "cidade/UF": cidade,
                    "estado": estado,
                    "data_coleta": datetime.today().strftime("%Y-%m-%d")
                })
            except Exception as e:
                print(f"Erro ao processar produto: {e}")
                continue
    print("---")

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()


Loja: Hiper Maceió JatiúcaMangabeiras | Cidade: Maceió/AL | Estado: AL
RegionId: None
---
Loja: Hipermercado Manaus AdrianópolisAdrianopolis | Cidade: Manaus/AM | Estado: AM
RegionId: None
---
Loja: Hipermercado Manaus ShoppingChapada | Cidade: nan | Estado: nan
RegionId: None
---
Loja: Hipermercado Ponta NegraDom Pedro | Cidade: nan | Estado: nan
RegionId: None
---
Loja: Big Bompreço ItabunaGoes Calmon | Cidade: Itabuna/BA | Estado: BA
RegionId: None
---


""


In [19]:
# Mapeamento de CEPs com base em nome da loja + cidade + estado
import pandas as pd
import requests
import time

# Carrega a planilha já ajustada
lojas = pd.read_excel("lista_lojas.xlsx")
lojas['cidade/UF'] = lojas['cidade/UF'].fillna(method='ffill')
lojas['estado'] = lojas['estado'].fillna(method='ffill')

# Função para extrair o CEP via Nominatim (OpenStreetMap)
def obter_cep_nominatim(endereco):
    try:
        url = f"https://nominatim.openstreetmap.org/search"
        params = {
            'q': endereco,
            'format': 'json',
            'addressdetails': 1,
            'limit': 1
        }
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, params=params, headers=headers, timeout=10)
        if r.status_code == 200:
            dados = r.json()
            if dados:
                return dados[0]['address'].get('postcode', None)
    except Exception as e:
        print(f"Erro ao buscar CEP para {endereco}: {e}")
    return None

# Criar campo com busca formatada
lojas[['cidade', 'uf']] = lojas['cidade/UF'].str.split('/', expand=True)
lojas['consulta_endereco'] = lojas.apply(
    lambda x: f"{x['cidade']}, {x['uf']}, Carrefour, Brasil", axis=1
)


# Aplicar a busca de CEP com limite de 10 primeiros (exemplo)
lojas_amostra = lojas.head(10).copy()
lojas_amostra['cep'] = lojas_amostra['consulta_endereco'].apply(lambda x: obter_cep_nominatim(x))
    
# Pausa para respeitar a API (Nominatim impõe limites de uso)
time.sleep(1)

# Visualizar resultado
lojas_amostra[['loja', 'cidade/UF', 'estado', 'cep']]


/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3184559279.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lojas['cidade/UF'] = lojas['cidade/UF'].fillna(method='ffill')
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3184559279.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lojas['estado'] = lojas['estado'].fillna(method='ffill')


,loja,cidade/UF,estado,cep
0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,None
1,Hipermercado Manaus AdrianópolisAdrianopolis,Manaus/AM,AM,None
2,Hipermercado Manaus ShoppingChapada,Manaus/AM,AM,None
3,Hipermercado Ponta NegraDom Pedro,Manaus/AM,AM,None
4,Big Bompreço ItabunaGoes Calmon,Itabuna/BA,BA,None
5,Carrefour Bairro Asa Norte IIAsa Norte,Brasília/DF,DF,None
6,Carrefour Bairro Bom MotivoLago Sul,Brasília/DF,DF,None
7,Hiper Águas ClarasÁguas Claras,Brasília/DF,DF,None
8,Hipermercado Brasilia Asa NorteAsa Norte,Brasília/DF,DF,None
9,Hipermercado Brasilia SulZona Industrial (Guará),Brasília/DF,DF,None


In [31]:
# Mapeamento de CEPs com base em coordenadas (latitude e longitude)
import pandas as pd
import requests
import time

# Carrega a planilha já ajustada
lojas = pd.read_excel("lista_lojas.xlsx")
lojas['cidade/UF'] = lojas['cidade/UF'].fillna(method='ffill')
lojas['estado'] = lojas['estado'].fillna(method='ffill')

# Função para obter CEP via coordenadas (geocodificação reversa)
def obter_cep_por_lat_long(lat, long):
    try:
        url = "https://nominatim.openstreetmap.org/reverse"
        params = {
            "format": "json",
            "lat": lat,
            "lon": long,
            "addressdetails": 1
        }
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, params=params, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            return data.get("address", {}).get("postcode", None)
    except Exception as e:
        print(f"Erro ao obter CEP para ({lat}, {long}): {e}")
    return None

# Aplicar a busca de CEP com limite de 10 primeiros (exemplo)
lojas_amostra = lojas.head(10).copy()
lojas_amostra['cep'] = lojas_amostra.apply(lambda row: obter_cep_por_lat_long(row['lat'], row['long']), axis=1)

# Pausa para respeitar a API (Nominatim impõe limites de uso)
time.sleep(1)

# Visualizar resultado
lojas_amostra[['loja', 'cidade/UF', 'estado', 'lat', 'long', 'cep']]

/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3666578831.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lojas['cidade/UF'] = lojas['cidade/UF'].fillna(method='ffill')
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3666578831.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lojas['estado'] = lojas['estado'].fillna(method='ffill')


,loja,cidade/UF,estado,lat,long,cep
0,Hiper Maceió JatiúcaMangabeiras,Maceió/AL,AL,-9.64959,-35.71739,57031-530
1,Hipermercado Manaus AdrianópolisAdrianopolis,Manaus/AM,AM,-3.10513,-60.01123,69000-000
2,Hipermercado Manaus ShoppingChapada,Manaus/AM,AM,-3.09326,-60.02191,69000-000
3,Hipermercado Ponta NegraDom Pedro,Manaus/AM,AM,-3.09195,-60.04980,69000-000
4,Big Bompreço ItabunaGoes Calmon,Itabuna/BA,BA,-14.79790,-39.27272,45600-006
5,Carrefour Bairro Asa Norte IIAsa Norte,Brasília/DF,DF,-15.77441,-47.88754,70737-000
6,Carrefour Bairro Bom MotivoLago Sul,Brasília/DF,DF,-15.84242,-47.87619,71635-170
7,Hiper Águas ClarasÁguas Claras,Brasília/DF,DF,-15.83163,-48.03967,71919-360
8,Hipermercado Brasilia Asa NorteAsa Norte,Brasília/DF,DF,-15.72217,-47.89789,71538-100
9,Hipermercado Brasilia SulZona Industrial (Guará),Brasília/DF,DF,-15.82938,-47.95478,71215-267


In [30]:
lojas_amostra

,estado,cidade/UF,loja,lat,long
0,AL,Maceió/AL,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739
1,AM,Manaus/AM,Hipermercado Manaus AdrianópolisAdrianopolis,-3.10513,-60.01123
2,AM,Manaus/AM,Hipermercado Manaus ShoppingChapada,-3.09326,-60.02191
3,AM,Manaus/AM,Hipermercado Ponta NegraDom Pedro,-3.09195,-60.04980
4,BA,Itabuna/BA,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272
5,DF,Brasília/DF,Carrefour Bairro Asa Norte IIAsa Norte,-15.77441,-47.88754
6,DF,Brasília/DF,Carrefour Bairro Bom MotivoLago Sul,-15.84242,-47.87619
7,DF,Brasília/DF,Hiper Águas ClarasÁguas Claras,-15.83163,-48.03967
8,DF,Brasília/DF,Hipermercado Brasilia Asa NorteAsa Norte,-15.72217,-47.89789
9,DF,Brasília/DF,Hipermercado Brasilia SulZona Industrial (Guará),-15.82938,-47.95478


In [23]:
# Coleta de Preços Carrefour por loja (Tilápia) usando CEP
import pandas as pd
import requests
from datetime import datetime
import time

# Carrega a planilha com coordenadas e CEPs já extraídos
df_lojas = pd.read_excel("lista_lojas.xlsx")
df_lojas['cidade/UF'] = df_lojas['cidade/UF'].fillna(method='ffill')
df_lojas['estado'] = df_lojas['estado'].fillna(method='ffill')

# Função para obter regionId a partir do CEP
def get_region_id_por_cep(cep):
    try:
        url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode={cep}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                return data[0].get("regionId")
    except Exception as e:
        print(f"Erro ao obter regionId para CEP {cep}: {e}")
    return None

# Função para buscar produtos com regionId
def buscar_produtos(region_id, termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    cookies = {"vtex_segment": f'{{"channel":"1","regionId":"{region_id}"}}'}
    try:
        r = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Selecionar primeiras 10 lojas
lojas_amostra = df_lojas.head(10).copy()
registros = []

# Iterar pelas lojas e coletar os produtos
for _, row in lojas_amostra.iterrows():
    cep = row.get('cep')
    if pd.notnull(cep):
        region_id = get_region_id_por_cep(cep)
        print(f"Loja: {row['loja']} | CEP: {cep} | RegionId: {region_id}")
        if region_id:
            produtos = buscar_produtos(region_id, termo="tilapia")
            print(f"Produtos encontrados: {len(produtos)}")
            for item in produtos:
                try:
                    nome = item.get("productName", "")
                    preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
                    registros.append({
                        "produto": nome,
                        "preco": preco,
                        "loja": row['loja'],
                        "cidade/UF": row['cidade/UF'],
                        "estado": row['estado'],
                        "cep": cep,
                        "data_coleta": datetime.today().strftime("%Y-%m-%d")
                    })
                except Exception as e:
                    print(f"Erro ao processar produto: {e}")
        time.sleep(1)  # respeitar limite de requisições

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()


/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/2811386486.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_lojas['cidade/UF'] = df_lojas['cidade/UF'].fillna(method='ffill')
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/2811386486.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_lojas['estado'] = df_lojas['estado'].fillna(method='ffill')


""


In [24]:
# Coleta de Preços Carrefour por loja (Tilápia) usando CEP
import pandas as pd
import requests
from datetime import datetime
import time

# Carrega a planilha com coordenadas e CEPs já extraídos
df_lojas = pd.read_excel("lista_lojas.xlsx")
df_lojas['cidade/UF'] = df_lojas['cidade/UF'].fillna(method='ffill')
df_lojas['estado'] = df_lojas['estado'].fillna(method='ffill')

# Função para obter regionId a partir do CEP
def get_region_id_por_cep(cep):
    try:
        url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode={cep}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                return data[0].get("regionId")
    except Exception as e:
        print(f"Erro ao obter regionId para CEP {cep}: {e}")
    return None

# Função para buscar produtos com regionId
def buscar_produtos(region_id, termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    cookies = {"vtex_segment": f'{{"channel":"1","regionId":"{region_id}"}}'}
    try:
        r = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Selecionar primeiras 10 lojas
lojas_amostra = df_lojas.head(10).copy()
registros = []

# Iterar pelas lojas e coletar os produtos
for _, row in lojas_amostra.iterrows():
    cep = row.get('cep')
    print(f"---\nLoja: {row['loja']}\nCEP: {cep}")
    if pd.notnull(cep):
        region_id = get_region_id_por_cep(cep)
        print(f"RegionId: {region_id}")
        if region_id:
            produtos = buscar_produtos(region_id, termo="tilapia")
            print(f"Produtos retornados: {len(produtos)}")
            for item in produtos:
                try:
                    nome = item.get("productName", "")
                    preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
                    print(f"Produto: {nome} | Preço: {preco}")
                    registros.append({
                        "produto": nome,
                        "preco": preco,
                        "loja": row['loja'],
                        "cidade/UF": row['cidade/UF'],
                        "estado": row['estado'],
                        "cep": cep,
                        "data_coleta": datetime.today().strftime("%Y-%m-%d")
                    })
                except Exception as e:
                    print(f"Erro ao processar produto: {e}")
        time.sleep(1)  # respeitar limite de requisições

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()


---
Loja: Hiper Maceió JatiúcaMangabeiras
CEP: None
---
Loja: Hipermercado Manaus AdrianópolisAdrianopolis
CEP: None
---
Loja: Hipermercado Manaus ShoppingChapada
CEP: None
---
Loja: Hipermercado Ponta NegraDom Pedro
CEP: None
---
Loja: Big Bompreço ItabunaGoes Calmon
CEP: None
---
Loja: Carrefour Bairro Asa Norte IIAsa Norte
CEP: None
---
Loja: Carrefour Bairro Bom MotivoLago Sul
CEP: None
---
Loja: Hiper Águas ClarasÁguas Claras
CEP: None
---
Loja: Hipermercado Brasilia Asa NorteAsa Norte
CEP: None
---
Loja: Hipermercado Brasilia SulZona Industrial (Guará)
CEP: None


/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3821153249.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_lojas['cidade/UF'] = df_lojas['cidade/UF'].fillna(method='ffill')
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3821153249.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_lojas['estado'] = df_lojas['estado'].fillna(method='ffill')


""


In [34]:
# Coleta de Preços Carrefour por loja (Tilápia) usando CEP
import pandas as pd
import requests
from datetime import datetime
import time

# Carrega a planilha com os CEPs preenchidos
df_lojas = pd.read_excel("lista_lojas_com_ceps.xlsx")
df_lojas['cidade/UF'] = df_lojas['cidade/UF'].ffill()
df_lojas['estado'] = df_lojas['estado'].ffill()

# Função para obter regionId a partir do CEP
def get_region_id_por_cep(cep):
    try:
        url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode={cep}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                return data[0].get("regionId")
    except Exception as e:
        print(f"Erro ao obter regionId para CEP {cep}: {e}")
    return None

# Função para buscar produtos com regionId
def buscar_produtos(region_id, termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    cookies = {"vtex_segment": f'{{"channel":"1","regionId":"{region_id}"}}'}
    try:
        r = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Selecionar primeiras 10 lojas
lojas_amostra = df_lojas.head(10).copy()
registros = []

# Iterar pelas lojas e coletar os produtos
for _, row in lojas_amostra.iterrows():
    cep = row.get('cep')
    print(f"---\nLoja: {row['loja']}\nCEP: {cep}")
    if pd.notnull(cep):
        region_id = get_region_id_por_cep(cep)
        print(f"RegionId: {region_id}")
        if region_id:
            produtos = buscar_produtos(region_id, termo="tilapia")
            print(f"Produtos retornados: {len(produtos)}")
            for item in produtos:
                try:
                    nome = item.get("productName", "")
                    preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
                    print(f"Produto: {nome} | Preço: {preco}")
                    registros.append({
                        "produto": nome,
                        "preco": preco,
                        "loja": row['loja'],
                        "cidade/UF": row['cidade/UF'],
                        "estado": row['estado'],
                        "cep": cep,
                        "data_coleta": datetime.today().strftime("%Y-%m-%d")
                    })
                except Exception as e:
                    print(f"Erro ao processar produto: {e}")
        time.sleep(1)  # respeitar limite de requisições

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()

---
Loja: Hiper Maceió JatiúcaMangabeiras
CEP: 57031-530
RegionId: None
---
Loja: Hipermercado Manaus AdrianópolisAdrianopolis
CEP: 69000-000
RegionId: None
---
Loja: Hipermercado Manaus ShoppingChapada
CEP: 69000-000
RegionId: None
---
Loja: Hipermercado Ponta NegraDom Pedro
CEP: 69000-000
RegionId: None
---
Loja: Big Bompreço ItabunaGoes Calmon
CEP: 45600-006
RegionId: None
---
Loja: Carrefour Bairro Asa Norte IIAsa Norte
CEP: 70737-000
RegionId: None
---
Loja: Carrefour Bairro Bom MotivoLago Sul
CEP: 71635-170
RegionId: None
---
Loja: Hiper Águas ClarasÁguas Claras
CEP: 71919-360
RegionId: None
---
Loja: Hipermercado Brasilia Asa NorteAsa Norte
CEP: 71538-100
RegionId: None
---
Loja: Hipermercado Brasilia SulZona Industrial (Guará)
CEP: 71215-267
RegionId: None


""


In [33]:
lojas_amostra.to_excel("lista_lojas_com_ceps.xlsx", index=False)


In [32]:
lojas_amostra  

,estado,cidade/UF,loja,lat,long,cep
0,AL,Maceió/AL,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739,57031-530
1,AM,Manaus/AM,Hipermercado Manaus AdrianópolisAdrianopolis,-3.10513,-60.01123,69000-000
2,AM,Manaus/AM,Hipermercado Manaus ShoppingChapada,-3.09326,-60.02191,69000-000
3,AM,Manaus/AM,Hipermercado Ponta NegraDom Pedro,-3.09195,-60.04980,69000-000
4,BA,Itabuna/BA,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272,45600-006
5,DF,Brasília/DF,Carrefour Bairro Asa Norte IIAsa Norte,-15.77441,-47.88754,70737-000
6,DF,Brasília/DF,Carrefour Bairro Bom MotivoLago Sul,-15.84242,-47.87619,71635-170
7,DF,Brasília/DF,Hiper Águas ClarasÁguas Claras,-15.83163,-48.03967,71919-360
8,DF,Brasília/DF,Hipermercado Brasilia Asa NorteAsa Norte,-15.72217,-47.89789,71538-100
9,DF,Brasília/DF,Hipermercado Brasilia SulZona Industrial (Guará),-15.82938,-47.95478,71215-267


In [35]:
https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode=70737-000


SyntaxError: invalid syntax (2438562811.py, line 1)

In [36]:
# Coleta de Preços Carrefour - Tilápia (sem uso de regionId)
import pandas as pd
import requests
from datetime import datetime

# Função para buscar produtos sem regionalização
def buscar_produtos_globais(termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Coletar produtos globais de "tilapia"
produtos = buscar_produtos_globais("tilapia")
registros = []

for item in produtos:
    try:
        nome = item.get("productName", "")
        sellers = item['items'][0].get('sellers', [])
        for seller in sellers:
            preco = seller['commertialOffer']['Price']
            registros.append({
                "produto": nome,
                "preco": preco,
                "seller_id": seller.get("sellerId"),
                "data_coleta": datetime.today().strftime("%Y-%m-%d")
            })
    except Exception as e:
        print(f"Erro ao processar item: {e}")

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()


,produto,preco,seller_id,data_coleta
0,Filé de Tilápia Fresco Carrefour 500 g,64.89,1,2025-04-18
1,Filé de Tilápia sem Espinha sem Pele Congelado...,65.99,1,2025-04-18
2,Filé de Tilápia a Milanesa Congelado Swift 500 g,42.25,1,2025-04-18
3,Filé de Tilápia Congelado Carrefour Special Sa...,21.75,1,2025-04-18
4,Quadritos de Peixe Empanado Costa Sul Pacote 400g,10.29,1,2025-04-18


In [37]:
# Coleta de Preços Carrefour - Tilápia (com seller_id e nome)
import pandas as pd
import requests
from datetime import datetime

# Função para buscar produtos sem regionalização
def buscar_produtos_globais(termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Coletar produtos globais de "tilapia"
produtos = buscar_produtos_globais("tilapia")
registros = []

for item in produtos:
    try:
        nome = item.get("productName", "")
        sellers = item['items'][0].get('sellers', [])
        for seller in sellers:
            preco = seller['commertialOffer']['Price']
            registros.append({
                "produto": nome,
                "preco": preco,
                "seller_id": seller.get("sellerId"),
                "seller_nome": seller.get("name"),
                "data_coleta": datetime.today().strftime("%Y-%m-%d")
            })
    except Exception as e:
        print(f"Erro ao processar item: {e}")

# Resultado final
resultado = pd.DataFrame(registros)
resultado.head()


,produto,preco,seller_id,seller_nome,data_coleta
0,Filé de Tilápia Fresco Carrefour 500 g,64.89,1,None,2025-04-18
1,Filé de Tilápia sem Espinha sem Pele Congelado...,65.99,1,None,2025-04-18
2,Filé de Tilápia a Milanesa Congelado Swift 500 g,42.25,1,None,2025-04-18
3,Filé de Tilápia Congelado Carrefour Special Sa...,21.75,1,None,2025-04-18
4,Quadritos de Peixe Empanado Costa Sul Pacote 400g,10.29,1,None,2025-04-18


In [38]:
# Mapeamento de sellers Carrefour por categoria de produto
import pandas as pd
import requests
from datetime import datetime

# Função para buscar produtos sem regionalização
def buscar_produtos_globais(termo):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos para '{termo}': {e}")
    return []

# Lista de termos a explorar
termos_busca = ["peixe", "tilapia", "arroz", "laticinio", "frango"]
mapeamento = []

# Coletar sellers distintos por categoria
for termo in termos_busca:
    produtos = buscar_produtos_globais(termo)
    for item in produtos:
        nome = item.get("productName", "")
        sellers = item['items'][0].get('sellers', [])
        for seller in sellers:
            mapeamento.append({
                "termo": termo,
                "produto": nome,
                "seller_id": seller.get("sellerId"),
                "seller_nome": seller.get("name"),
                "preco": seller['commertialOffer']['Price'],
                "data_coleta": datetime.today().strftime("%Y-%m-%d")
            })

# Converter em DataFrame
df_mapeamento = pd.DataFrame(mapeamento)
df_mapeamento.head()


,termo,produto,seller_id,seller_nome,preco,data_coleta
0,peixe,Peixe Inteiro Cavalinha Congelado Planalto 800 g,1,None,14.25,2025-04-18
1,peixe,Peixe Sardinha Planalto Fish Congelado 800 g,1,None,10.29,2025-04-18
2,peixe,Peixe Salgado Pedaço Resfriado Carrefour Aprox...,1,None,50.49,2025-04-18
3,peixe,Peixe Desfiado Dessalgado Polaca do Alasca Rib...,1,None,20.55,2025-04-18
4,peixe,Casquinha de Peixe Swift 330 g,1,None,30.90,2025-04-18


In [40]:
# Coleta de Preços com Selenium - Carrefour (10 cidades)
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import pandas as pd
import re

# Data atual
data_de_hoje = datetime.date.today()

# URL base do Carrefour
carrefour = "https://mercado.carrefour.com.br/s?q=kani&sort=score_desc&page=0"

# Lista para armazenar resultados
precos_carrefour = []

# Laço para visitar 10 cidades
j = 1
while j < 10:
    print(f"\n🟦 Coletando cidade {j}...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(carrefour)
    time.sleep(4)

    try:
        # Abrir seletor de cidade
        driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
        time.sleep(2)
        driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
        time.sleep(4)

        # Selecionar cidade
        select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))
        if not select.options[j].is_enabled():
            print(f"Cidade {j} desabilitada. Pulando...")
            j += 1
            driver.quit()
            continue
        select.select_by_index(j)
        time.sleep(4)

        # Selecionar loja
        botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
        loja, _, cidade, _ = botao_cidade[0].text.split("\n")
        botao_cidade[0].click()
        time.sleep(5)

        # Scroll para carregar produtos
        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)

        # Extrair produtos visíveis
        divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")
        for div in divs_carrefour:
            dtxt = div.text.strip()
            if len(dtxt) > 0:
                # Limpar estrutura usando regex
                preco_match = re.search(r"R\$\s?\d{1,3}(\.\d{3})*,\d{2}", dtxt)
                preco = preco_match.group(0) if preco_match else ""
                nome = dtxt.split("\n")[0]  # primeira linha geralmente é o nome
                precos_carrefour.append({
                    "produto": nome,
                    "preco": preco,
                    "loja": loja.strip(),
                    "cidade": cidade.strip(),
                    "data": data_de_hoje
                })
    except Exception as e:
        print(f"Erro em cidade {j}: {e}")

    j += 1
    driver.quit()
    time.sleep(2)

# Gerar DataFrame
resultado = pd.DataFrame(precos_carrefour)
resultado.head()


<>:32: SyntaxWarning: invalid escape sequence '\['
<>:32: SyntaxWarning: invalid escape sequence '\['
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/3062360448.py:32: SyntaxWarning: invalid escape sequence '\['
  driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()



🟦 Coletando cidade 1...
Cidade 1 desabilitada. Pulando...

🟦 Coletando cidade 2...

🟦 Coletando cidade 3...
Erro em cidade 3: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000105202e10 cxxbridge1$str$ptr + 2817040
1   chromedriver                        0x00000001051fb0ac cxxbridge1$str$ptr + 2784940
2   chromedriver                        0x0000000104d428d8 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104d896a0 cxxbridge1$string$len + 383276
4   chromedriver                        0x0000000104dca7b8 cxxbridge1$string$len + 649796
5   chromedriver                      

,produto,preco,loja,cidade,data
0,Kani Congelado Frescatto 200g,"R$ 13,39",Hiper Maceió Jatiúca,"Mangabeiras, Maceió - AL",2025-04-18
1,Kani Congelado Frescatto 200g,"R$ 12,59",Hipermercado Brasilia Sul,"Zona Industrial (Guará), Brasília - DF",2025-04-18
2,Kani Congelado Frescatto 200g,"R$ 12,19",Hipermercado Brasilia Taguatinga,"Aguas Claras, Taguatinga - DF",2025-04-18
3,Kani Congelado Costa Sul 200g,"R$ 13,39",Hipermercado Vila Velha Shopping,"Divino Espirito Santo, Vila Velha - ES",2025-04-18
4,Kani Congelado Frescatto 200g,"R$ 15,99",Hipermercado Vila Velha Shopping,"Divino Espirito Santo, Vila Velha - ES",2025-04-18


In [41]:
# Coleta de endereços das lojas Carrefour via Selenium
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Lista para armazenar resultados
enderecos_lojas = []

# URL base com busca genérica para ativar interface
carrefour = "https://mercado.carrefour.com.br/s?q=kani"

# Loop para as 10 primeiras cidades
j = 1
while j < 10:
    print(f"\nColetando loja da cidade {j}...")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(carrefour)
    time.sleep(4)

    try:
        # Abrir seletor de cidade
        driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
        time.sleep(2)
        driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
        time.sleep(4)

        # Selecionar cidade
        select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))
        if not select.options[j].is_enabled():
            print(f"Cidade {j} desabilitada. Pulando...")
            j += 1
            driver.quit()
            continue
        select.select_by_index(j)
        time.sleep(4)

        # Selecionar loja e capturar info
        botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
        loja_texto = botao_cidade[0].text.strip().split("\n")
        if len(loja_texto) >= 3:
            nome_loja = loja_texto[0].strip()
            endereco = loja_texto[1].strip()
            cidade_estado = loja_texto[2].strip()
            enderecos_lojas.append({
                "loja": nome_loja,
                "endereco": endereco,
                "cidade_estado": cidade_estado
            })
        else:
            print("Formato de loja inesperado.")
    except Exception as e:
        print(f"Erro em cidade {j}: {e}")

    j += 1
    driver.quit()
    time.sleep(1)

# Converter para DataFrame
df_lojas = pd.DataFrame(enderecos_lojas)
df_lojas.head()


<>:27: SyntaxWarning: invalid escape sequence '\['
<>:27: SyntaxWarning: invalid escape sequence '\['
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_18099/2568103212.py:27: SyntaxWarning: invalid escape sequence '\['
  driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()



Coletando loja da cidade 1...
Cidade 1 desabilitada. Pulando...

Coletando loja da cidade 2...

Coletando loja da cidade 3...

Coletando loja da cidade 4...

Coletando loja da cidade 5...

Coletando loja da cidade 6...

Coletando loja da cidade 7...

Coletando loja da cidade 8...

Coletando loja da cidade 9...


,loja,endereco,cidade_estado
0,Hiper Maceió Jatiúca,"Avenida Comendador Gustavo Paiva, 2650 A","Mangabeiras, Maceió - AL"
1,Big Bompreço Itabuna,"Avenida Aziz Maron, S/N - FRENTE PAR","Goes Calmon, Itabuna - BA"
2,Hipermercado Manaus Shopping,Av Djalma Batista 482 Loja B,"Chapada, Manaus - AM"
3,Hipermercado Brasilia Sul,SMAS Trecho (Carrefour) S/N,"Zona Industrial (Guará), Brasília - DF"
4,Hipermercado Brasilia Taguatinga,Qs 03 Rua 420 Area Especial Lt 2,"Aguas Claras, Taguatinga - DF"


In [42]:
# Função para obter CEP via Nominatim
def obter_cep_nominatim(endereco_completo):
    try:
        url = "https://nominatim.openstreetmap.org/search"
        params = {
            'q': endereco_completo,
            'format': 'json',
            'addressdetails': 1,
            'limit': 1
        }
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, params=params, headers=headers, timeout=10)
        if r.status_code == 200:
            dados = r.json()
            if dados:
                return dados[0]['address'].get('postcode', None)
    except Exception as e:
        print(f"Erro ao buscar CEP para {endereco_completo}: {e}")
    return None

# Gerar coluna de consulta e aplicar para obter CEP
df_lojas['consulta'] = df_lojas['endereco'] + ", " + df_lojas['cidade_estado'] + ", Brasil"
df_lojas['cep'] = df_lojas['consulta'].apply(lambda x: obter_cep_nominatim(x))
df_lojas.drop(columns=['consulta'], inplace=True)

df_lojas.head()


,loja,endereco,cidade_estado,cep
0,Hiper Maceió Jatiúca,"Avenida Comendador Gustavo Paiva, 2650 A","Mangabeiras, Maceió - AL",57025-640
1,Big Bompreço Itabuna,"Avenida Aziz Maron, S/N - FRENTE PAR","Goes Calmon, Itabuna - BA",None
2,Hipermercado Manaus Shopping,Av Djalma Batista 482 Loja B,"Chapada, Manaus - AM",None
3,Hipermercado Brasilia Sul,SMAS Trecho (Carrefour) S/N,"Zona Industrial (Guará), Brasília - DF",None
4,Hipermercado Brasilia Taguatinga,Qs 03 Rua 420 Area Especial Lt 2,"Aguas Claras, Taguatinga - DF",None


In [45]:
mascarados = df_lojas['cep'].isna()
df_lojas.loc[mascarados, 'cep'] = df_lojas.loc[mascarados, 'consulta'].apply(lambda x: obter_cep_nominatim(x))


df_lojas

,loja,endereco,cidade_estado,cep,endereco_limpo,consulta
0,Hiper Maceió Jatiúca,"Avenida Comendador Gustavo Paiva, 2650 A","Mangabeiras, Maceió - AL",57025-640,"Avenida Comendador Gustavo Paiva, 2650 A","Avenida Comendador Gustavo Paiva, 2650 A, Mang..."
1,Big Bompreço Itabuna,"Avenida Aziz Maron, S/N - FRENTE PAR","Goes Calmon, Itabuna - BA",45600-190,"Avenida Aziz Maron,","Avenida Aziz Maron, , Goes Calmon, Itabuna - B..."
2,Hipermercado Manaus Shopping,Av Djalma Batista 482 Loja B,"Chapada, Manaus - AM",None,Av Djalma Batista 482 Loja B,"Av Djalma Batista 482 Loja B, Chapada, Manaus ..."
3,Hipermercado Brasilia Sul,SMAS Trecho (Carrefour) S/N,"Zona Industrial (Guará), Brasília - DF",None,SMAS Trecho (Carrefour),"SMAS Trecho (Carrefour) , Guará, Brasília - DF..."
4,Hipermercado Brasilia Taguatinga,Qs 03 Rua 420 Area Especial Lt 2,"Aguas Claras, Taguatinga - DF",None,Qs 03 Rua 420 Area Especial Lt 2,"Qs 03 Rua 420 Area Especial Lt 2, Águas Claras..."
5,Hipermercado Vila Velha Shopping,Rua Luciano Das Neves 2418,"Divino Espirito Santo, Vila Velha - ES",None,Rua Luciano Das Neves 2418,"Rua Luciano Das Neves 2418, Divino Espirito Sa..."
6,Hipermercado Goiânia Anápolis,Av Brasil 505 Loja 1,"Cidade Jardim, Anapolis - GO",None,Av Brasil 505 Loja 1,"Av Brasil 505 Loja 1, Cidade Jardim, Anapolis ..."
7,Hipermercado Goiânia Sul,Av Deputado Jamel Cecilio 3900,"Jd Goias, Goiania - GO",None,Av Deputado Jamel Cecilio 3900,"Av Deputado Jamel Cecilio 3900, Jd Goias, Goia..."


In [47]:
# Coleta de Preços por CEP e RegionId no Carrefour
import pandas as pd
import requests
from datetime import datetime
import time

# Carregar o DataFrame com CEPs já coletados
#df_lojas = pd.read_excel("lista_lojas_com_cep.xlsx")  # ajuste se estiver em outro nome
df_lojas 
# Filtrar apenas os CEPs válidos
df_validos = df_lojas[df_lojas['cep'].notna()].copy()

# Função para obter regionId a partir do CEP
def get_region_id_por_cep(cep):
    try:
        url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode={cep}"
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                return data[0].get("regionId")
    except Exception as e:
        print(f"Erro ao obter regionId para CEP {cep}: {e}")
    return None

# Função para buscar produtos com regionId
def buscar_produtos(region_id, termo="tilapia"):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    cookies = {"vtex_segment": f'{{"channel":"1","regionId":"{region_id}"}}'}
    try:
        r = requests.get(url, headers=headers, cookies=cookies, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos: {e}")
    return []

# Lista de resultados
registros = []

# Loop sobre lojas com CEPs
for _, row in df_validos.iterrows():
    cep = row['cep']
    loja = row['loja']
    cidade = row['cidade_estado']
    endereco = row['endereco']

    print(f"\n🔍 Buscando RegionId para CEP {cep} ({loja})")
    region_id = get_region_id_por_cep(cep)
    print(f"RegionId: {region_id}")

    if region_id:
        produtos = buscar_produtos(region_id, termo="tilapia")
        print(f"Produtos encontrados: {len(produtos)}")

        for item in produtos:
            try:
                nome = item.get("productName", "")
                preco = item['items'][0]['sellers'][0]['commertialOffer']['Price']
                registros.append({
                    "produto": nome,
                    "preco": preco,
                    "loja": loja,
                    "cep": cep,
                    "region_id": region_id,
                    "cidade_estado": cidade,
                    "endereco": endereco,
                    "data": datetime.today().strftime("%Y-%m-%d")
                })
            except Exception as e:
                print(f"Erro ao processar produto: {e}")
    time.sleep(1)

# Converter em DataFrame
resultado = pd.DataFrame(registros)
resultado.head()



🔍 Buscando RegionId para CEP 57025-640 (Hiper Maceió Jatiúca)
Erro ao obter regionId para CEP 57025-640: HTTPSConnectionPool(host='mercado.carrefour.com.br', port=443): Read timed out. (read timeout=10)
RegionId: None

🔍 Buscando RegionId para CEP 45600-190 (Big Bompreço Itabuna)
RegionId: None


""


In [48]:
# Coleta de Lojas Carrefour via https://www.carrefour.com.br/localizador-de-lojas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Inicializa navegador
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = "https://www.carrefour.com.br/localizador-de-lojas"
driver.get(url)
time.sleep(5)

# Carregar todas as lojas (clicar no "Mostrar mais lojas")
while True:
    try:
        botao = driver.find_element(By.XPATH, "//button[contains(text(), 'Mostrar mais lojas')]")
        botao.click()
        time.sleep(2)
    except:
        break

# Extrair blocos de loja
cards = driver.find_elements(By.CSS_SELECTOR, "div.StoreCard_storeCardContent__ymw5w")

dados_lojas = []
for card in cards:
    try:
        nome = card.find_element(By.TAG_NAME, "h2").text.strip()
        linhas = card.text.split("\n")
        endereco = linhas[1].strip() if len(linhas) > 1 else ""
        cidade_estado = linhas[2].strip() if len(linhas) > 2 else ""
        cep = ""
        for l in linhas:
            if "CEP:" in l:
                cep = l.split("CEP:")[-1].strip()
        dados_lojas.append({
            "loja": nome,
            "endereco": endereco,
            "cidade_estado": cidade_estado,
            "cep": cep
        })
    except Exception as e:
        print(f"Erro ao processar card: {e}")

# Fechar navegador
driver.quit()

# Criar DataFrame
df_lojas_localizador = pd.DataFrame(dados_lojas)
df_lojas_localizador.head()



""


In [52]:
import pandas as pd
import requests
import time

# Carrega a planilha de lojas Carrefour
df_lojas = pd.read_csv("lojas_carrefour_2025.csv")

# Adiciona coluna regionId
df_lojas["regionId"] = None

# Função para buscar regionId a partir do CEP
def buscar_region_id(cep):
    try:
        url = f"https://mercado.carrefour.com.br/api/checkout/pub/regions/?country=BRA&postalCode={cep}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            dados = response.json()
            if isinstance(dados, list) and len(dados) > 0:
                return dados[0].get("regionId", None)
    except Exception as e:
        return None
    return None

# Loop com busca para cada loja com CEP
for i, row in df_lojas.iterrows():
    cep = row["cep"]
    if pd.notna(cep):
        region_id = buscar_region_id(cep)
        df_lojas.at[i, "regionId"] = region_id
        time.sleep(0.5)

# Exibe os resultados com regionId não nulo
df_lojas_validas = df_lojas[df_lojas["regionId"].notna()].copy()

print(df_lojas[df_lojas["regionId"].notna()].head())  # Para ver os primeiros resultados



Empty DataFrame
Columns: [loja, endereco, cidade_estado, cep, tipo_loja, regionId]
Index: []


In [53]:
produtos = buscar_produtos()
for item in produtos:
    for seller in item['items'][0]['sellers']:
        print(seller['sellerId'], seller['commertialOffer']['Price'])


TypeError: buscar_produtos() missing 1 required positional argument: 'region_id'

In [54]:
# Coleta de produtos e mapeamento de seller_id no Carrefour
import requests
import pandas as pd
from datetime import datetime
import time

# Lista de termos a pesquisar
termos_busca = ["tilapia", "kani", "peixe", "frango", "carne"]

# Função para buscar produtos por termo (consulta geral)
def buscar_produtos(termo):
    url = f"https://mercado.carrefour.com.br/api/catalog_system/pub/products/search?ft={termo}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://mercado.carrefour.com.br/",
        "Origin": "https://mercado.carrefour.com.br"
    }
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code in [200, 206]:
            return r.json()
    except Exception as e:
        print(f"Erro ao buscar produtos para '{termo}': {e}")
    return []

# Coleta dos dados
registros = []

for termo in termos_busca:
    produtos = buscar_produtos(termo)
    print(f"\n🔍 Termo: {termo} | Produtos encontrados: {len(produtos)}")
    for item in produtos:
        nome = item.get("productName", "")
        product_id = item.get("productId", "")
        for seller in item["items"][0]["sellers"]:
            registros.append({
                "termo": termo,
                "produto": nome,
                "product_id": product_id,
                "seller_id": seller.get("sellerId"),
                "preco": seller.get("commertialOffer", {}).get("Price"),
                "data": datetime.today().strftime("%Y-%m-%d")
            })
    time.sleep(1)

# Criar DataFrame e mostrar primeiros resultados
df_sellers = pd.DataFrame(registros)
df_sellers.sort_values(by=["seller_id", "termo"]).drop_duplicates().head()


🔍 Termo: tilapia | Produtos encontrados: 10

🔍 Termo: kani | Produtos encontrados: 10

🔍 Termo: peixe | Produtos encontrados: 10

🔍 Termo: frango | Produtos encontrados: 10

🔍 Termo: carne | Produtos encontrados: 10


,termo,produto,product_id,seller_id,preco,data
40,carne,PANQUECA CARNE kg,4719112,1,62.90,2025-04-18
41,carne,Mini Esfihas de Carne Artesanal Massas 400g,15437,1,30.85,2025-04-18
42,carne,Macarrão Instantâneo Lámen com Tempero Carne R...,4788936,1,1.85,2025-04-18
43,carne,Ração Úmida para Gatos Adulto Fancy Feast Cass...,7005975,1,5.99,2025-04-18
44,carne,Ração Úmida para Gatos Adulto Fancy Feast Peti...,7005976,1,6.99,2025-04-18
